In [14]:
import spacy
import pandas as pd
nlp = spacy.load('en_core_web_md')

In [9]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer']

for disabling the components of a pipeline:
```python
for doc in nlp.pipe("blah blah blah", disable=["tagger", "parser", "lemmatizer"]):
    pass
```

In [96]:
df = pd.read_csv('/home/iamauser/Downloads/stack_overflow_ds/archive/Questions.csv', encoding='ISO-8859-1', usecols=['Title','Id'],nrows=1000000)

In [142]:
def getPatterns(langname):
    return[
        [{"LOWER":{"REGEX":'(%s\d+\.?\d*.?\d*)' % langname}}],
        [{"LOWER":"%s"%langname},{"TEXT":{"REGEX":'(\d+\.?\d*.?\d*)'}}]
    ]

In [145]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)
# python_pat1 = [{"LOWER":"python"}]
c_pattern = [{"LOWER":"c"}]
c_plus = [{"LOWER":"c++"}]
c_sharp = [{"LOWER":"c#"}]
sql_pattern =[{"LOWER":"sql"}]
go_pattern = [{"LOWER":"go", "POS":{"NOT_IN":['VERB']}}]
go_pattern1 = [{"LOWER":"golang"}]
pattern_list = getPatterns('python') + getPatterns('ruby') + getPatterns('java') + \
getPatterns('javascript') + getPatterns('js') + [c_pattern] + [c_plus] + [c_sharp] + [sql_pattern] +\
[go_pattern] + [go_pattern1]


# matcher.add("PROGLANG",getPatterns('SQL'))
matcher.add("PROGLANG",pattern_list)
title_gen = (title for title in df['Title'])

test_title_gen = (next(title_gen) for _ in range(100000)) #insert the dataset size in range

matcher(nlp('python'))


[]

In [117]:
dataset = []
        
def get_training_entry(doc):
    matched_elems = matcher(doc)
#     print(len(dataset), end=", ")
    if len(matched_elems)!=0:
        detections = [(doc[start:end].start_char,doc[start:end].end_char,doc.vocab.strings[hash_id]) for hash_id,start,end in matched_elems]
        return doc.text, {'entities':detections}
    else:
        return None
    
_ = [dataset.append(t_entry) for t_entry in [get_training_entry(doc) for doc in nlp.pipe(test_title_gen)] if t_entry is not None]
        

In [128]:
#Creating db for custom training 
from spacy.tokens import DocBin

nlp = spacy.blank("en")

db = DocBin()

for text,annot in dataset:
    doc = nlp.make_doc(text)
    ents = []
    for start,end,label in annot['entities']:
        span = doc.char_span(start, end, label=label,alignment_mode="contract")
        if span is None:
            print('skipping entry')
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)
doc

ValueError: [E1010] Unable to set entity information for token 2 which is included in more than one span in entities, blocked, missing or outside.